# Room Price Prediction System
This notebook allows you to predict room prices based on room details using pre-trained models.

## 1. Load Required Libraries and Models
First, we'll import necessary libraries and load our pre-trained model and preprocessing components.

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Load the pre-trained model and preprocessing components
try:
    model = joblib.load('models/room_price_model.pkl')
    scaler = joblib.load('models/feature_scaler.pkl')
    encoder = joblib.load('models/categorical_encoder.pkl')
except FileNotFoundError:
    print("Error: Required model files not found. Please ensure the model files are in the correct location.")

## 2. Create Input Interface
Define a function to collect room details from user input.

In [ ]:
def get_room_details():
    """
    Collect room details from user input
    """
    room_details = {}
    
    # Collect basic room details
    room_details['room_size'] = float(input("Enter room size (in square meters): "))
    room_details['floor_level'] = int(input("Enter floor level: "))
    room_details['view_type'] = input("Enter view type (city/ocean/garden): ")
    room_details['bed_type'] = input("Enter bed type (single/double/king): ")
    room_details['has_balcony'] = input("Has balcony? (yes/no): ").lower() == 'yes'
    room_details['has_wifi'] = input("Has WiFi? (yes/no): ").lower() == 'yes'
    room_details['season'] = input("Enter season (summer/winter/spring/fall): ")
    
    return room_details

## 3. Data Preprocessing
Transform the input data to match the format used during model training.

In [ ]:
def preprocess_input(room_details):
    """
    Preprocess the input data to match training data format
    """
    # Convert dictionary to DataFrame
    input_df = pd.DataFrame([room_details])
    
    # Separate numerical and categorical columns
    numerical_cols = ['room_size', 'floor_level']
    categorical_cols = ['view_type', 'bed_type', 'season']
    binary_cols = ['has_balcony', 'has_wifi']
    
    # Scale numerical features
    input_df[numerical_cols] = scaler.transform(input_df[numerical_cols])
    
    # Encode categorical features
    encoded_cats = encoder.transform(input_df[categorical_cols])
    encoded_df = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_cols))
    
    # Combine all features
    final_input = pd.concat([
        input_df[numerical_cols],
        encoded_df,
        input_df[binary_cols].astype(int)
    ], axis=1)
    
    return final_input

## 4. Price Prediction
Use the model to predict room price based on the processed input.

In [ ]:
def predict_price(processed_input):
    """
    Predict room price using the loaded model
    """
    prediction = model.predict(processed_input)
    
    # Calculate confidence interval (if using a model that supports it)
    try:
        confidence = model.predict_proba(processed_input)
        return prediction[0], confidence
    except:
        return prediction[0], None

## 5. Display Results
Show the predicted price and additional information.

In [ ]:
def display_results(price, confidence=None):
    """
    Display the prediction results
    """
    print("\n=== Price Prediction Results ===")
    print(f"Predicted Room Price: ${price:.2f}")
    
    if confidence is not None:
        print(f"Confidence Level: {confidence:.2%}")
    
    # Add some basic insights
    if price > 300:
        print("This is a premium price point")
    elif price > 200:
        print("This is a mid-range price point")
    else:
        print("This is a budget-friendly price point")

## Run the Complete Prediction Process
Execute all steps to get the room price prediction.

In [ ]:
# Main execution
if __name__ == "__main__":
    # Get room details from user
    print("Please enter room details:")
    room_details = get_room_details()
    
    # Preprocess the input
    processed_input = preprocess_input(room_details)
    
    # Make prediction
    predicted_price, confidence = predict_price(processed_input)
    
    # Display results
    display_results(predicted_price, confidence)